In [1]:
!pip install -U opencv-python tensorflow scikit-learn pandas matplotlib tensorflow_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 241.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.6/620.6 MB 156.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 243.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 225.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 184.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 175.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 202.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 211.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 192.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 188.0 MB/s eta 0:00:0

# IMPORTATION DES LIBRAIRIES

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets.public_api as tfds
import requests
import zipfile
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

2025-08-15 14:03:16.775500: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-15 14:03:16.836470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-15 14:03:18.259083: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Chargement du dataset

In [3]:
def telecharger_dezip(url, chemin_sauv="plant_village_dataset.zip", extract_path="."):
    print(" Début du téléchargement")
    try:
        response=requests.get(url, stream=True)
        response.raise_for_status()

        #Taille totale du fichier pour la barre de progression
        total_size=int(response.headers.get('content-length',0))
        block_size=1064
        bar_progression = tqdm(total=total_size, unit='iB', unit_scale=True)

        #Téléchargement
        with open(chemin_sauv, 'wb') as file:
            for data in response.iter_content(block_size):
                bar_progression.update(len(data))
                file.write(data)
        bar_progression.close()

        if total_size != 0 and bar_progression.n != total_size:
            print("ERREUR, quelque chose s'est mal passé pendant le téléchargement.")
            return

        print(f"Téléchargement terminé. Fichier sauvegardé sous : {chemin_sauv}")

        # Créer le dossier d'extraction s'il n'existe pas
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)

        # Décompresser le fichier ZIP
        print(f"Décompression du fichier dans le dossier : {extract_path}")
        with zipfile.ZipFile(chemin_sauv, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        print("Décompression terminée.")

        # Optionnel : Supprimer le fichier .zip après extraction pour économiser de l'espace
        print(f"Suppression du fichier {chemin_sauv}...")
        os.remove(chemin_sauv)
        print("Opération terminée avec succès !")

    except requests.exceptions.RequestException as e:
        print(f"Une erreur de réseau est survenue: {e}")
    except zipfile.BadZipFile:
        print("Erreur: Le fichier téléchargé n'est pas un fichier ZIP valide.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue: {e}")

In [4]:
URL = "https://data.mendeley.com/datasets/tywbtsjrjv/1/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/file_downloaded"

In [5]:
extract_folder = "plant_village_dataset"

In [6]:
telecharger_dezip(URL, "PlantVillage.zip", extract_folder)

 Début du téléchargement


100%|██████████| 949M/949M [00:49<00:00, 19.0MiB/s]  


Téléchargement terminé. Fichier sauvegardé sous : PlantVillage.zip
Décompression du fichier dans le dossier : plant_village_dataset
Décompression terminée.
Suppression du fichier PlantVillage.zip...
Opération terminée avec succès !


In [7]:
path="/workspace/plant_village_dataset/Plant_leave_diseases_dataset_with_augmentation"

In [8]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TEMPERATURE = 2.0

In [9]:
data_gen=ImageDataGenerator(rescale=1./255)

In [10]:
data=data_gen.flow_from_directory(
    path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 61486 images belonging to 39 classes.


# MODELISATION

## TS avec 

**Teacher:resnet50**

**Student:efficientnet_b0**


In [11]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F

In [12]:
# --------- 1. Préparer les données ---------
filepaths = []
labels = []
folds = os.listdir(path)
for fold in folds:
    f_path = os.path.join(path, fold)
    if not os.path.isdir(f_path):
        continue
    for file in os.listdir(f_path):
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

df = pd.DataFrame({'filepaths': filepaths, 'labels': labels})
print(f"Total des images trouvées : {len(df)}")

Total des images trouvées : 61486


In [13]:
# Split 80/20 avec stratification
train_df, test_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42,
    stratify=df['labels']
)
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df['labels']
)

In [14]:
# Mapping des classes en indices
class_names = sorted(df['labels'].unique())
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
num_classes = len(class_names)

In [15]:
# --------- 2. Dataset personnalisé ---------
class CustomImageDataset(Dataset):
    def __init__(self, df, class_to_idx, transform=None):
        self.df = df.reset_index(drop=True)
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepaths']
        label_name = self.df.loc[idx, 'labels']
        label = self.class_to_idx[label_name]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [16]:
# --------- 3. Data augmentation et loaders ---------
train_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # EfficientNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [17]:
train_dataset = CustomImageDataset(train_df, class_to_idx, transform=train_transforms)
val_dataset = CustomImageDataset(val_df, class_to_idx, transform=val_transforms)
test_dataset = CustomImageDataset(test_df, class_to_idx, transform=val_transforms)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU Name: NVIDIA L40S


In [ ]:
# Modèle Teacher (Professeur) - ResNet50
teacher_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
for param in teacher_model.parameters():
    param.requires_grad = False
num_ftrs_teacher = teacher_model.fc.in_features
teacher_model.fc = nn.Linear(num_ftrs_teacher, num_classes)
teacher_model = teacher_model.to(device)

# Modèle Student (Étudiant) 
student_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
num_ftrs_student = student_model.classifier[1].in_features
student_model.classifier[1] = nn.Linear(num_ftrs_student, num_classes)
student_model = student_model.to(device)

In [24]:
criterion_hard = nn.CrossEntropyLoss()

In [25]:
# ---------- Distillation Loss ----------
def distillation_loss(student_outputs, teacher_outputs, temperature, device):
    teacher_outputs = teacher_outputs.detach()
    soft_teacher_outputs = nn.functional.softmax(teacher_outputs / temperature, dim=1)
    soft_student_outputs = nn.functional.log_softmax(student_outputs / temperature, dim=1)
    dist_loss = nn.functional.kl_div(soft_student_outputs, soft_teacher_outputs, reduction='batchmean')
    return dist_loss * (temperature * temperature)


In [26]:
import torch.optim as optim

In [27]:
optimizer_student = optim.Adam(student_model.parameters(), lr=0.001)
scheduler_student = optim.lr_scheduler.StepLR(optimizer_student, step_size=7, gamma=0.1)

In [30]:
def train_teacher_student(teacher_model, student_model, criterion_hard, optimizer_student, scheduler_student, num_epochs, train_loader, val_loader, device):
    best_val_accuracy = 0.0
    best_model_path = '/workspace/models/best_student_efficientnetb0_model.pth'
    
    for epoch in range(num_epochs):
        student_model.train()
        teacher_model.eval()
        
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer_student.zero_grad()
            
            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)
            
            student_outputs = student_model(inputs)
            
            hard_loss = criterion_hard(student_outputs, labels)
            soft_loss = distillation_loss(student_outputs, teacher_outputs, TEMPERATURE, device)
            
            loss = hard_loss * 0.5 + soft_loss * 0.5
            
            loss.backward()
            optimizer_student.step()
            
            _, preds = torch.max(student_outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        scheduler_student.step()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        # Validation
        student_model.eval()
        val_loss = 0.0
        val_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student_model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion_hard(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} | '
              f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(student_model.state_dict(), best_model_path)
            print(f'Meilleur modèle étudiant sauvegardé avec une précision de validation de {best_val_accuracy:.4f}')


In [31]:
NUM_EPOCHS = 20
train_teacher_student(teacher_model, student_model, criterion_hard, optimizer_student, scheduler_student, NUM_EPOCHS, train_loader, val_loader, device)

Validation Epoch 1/20: 100%|██████████| 346/346 [00:19<00:00, 17.55it/s]


Epoch 1/20 - Train Loss: 0.5906 Acc: 0.9416 | Val Loss: 0.5068 Acc: 0.9757
Meilleur modèle étudiant sauvegardé avec une précision de validation de 0.9757


Validation Epoch 2/20: 100%|██████████| 346/346 [00:17<00:00, 19.26it/s]


Epoch 2/20 - Train Loss: 0.5636 Acc: 0.9523 | Val Loss: 0.4749 Acc: 0.9816
Meilleur modèle étudiant sauvegardé avec une précision de validation de 0.9816


Validation Epoch 3/20: 100%|██████████| 346/346 [00:17<00:00, 19.69it/s]


...

Epoch 18/20 - Train Loss: 0.4789 Acc: 0.9899 | Val Loss: 0.4080 Acc: 0.9973
Meilleur modèle étudiant sauvegardé avec une précision de validation de 0.9973


Validation Epoch 19/20: 100%|██████████| 346/346 [00:16<00:00, 20.80it/s]


Epoch 19/20 - Train Loss: 0.4783 Acc: 0.9904 | Val Loss: 0.4036 Acc: 0.9975
Meilleur modèle étudiant sauvegardé avec une précision de validation de 0.9975


Validation Epoch 20/20: 100%|██████████| 346/346 [00:16<00:00, 20.95it/s]

Epoch 20/20 - Train Loss: 0.4776 Acc: 0.9908 | Val Loss: 0.3977 Acc: 0.9974


#### Evaluation

In [32]:
import time, psutil, torch
from sklearn.metrics import classification_report

In [33]:
print("\nFormation du modèle étudiant terminée. Chargement du meilleur modèle pour l'évaluation.")
student_model.load_state_dict(torch.load('/workspace/models/best_student_efficientnetb0_model.pth'))
student_model.eval()


Formation du modèle étudiant terminée. Chargement du meilleur modèle pour l'évaluation.


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [34]:
all_preds = []
all_labels = []

In [35]:
start_time = time.time()
with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        batch_start = time.time()
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = student_model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # --- Profiling ---
        cpu_usage = psutil.cpu_percent(interval=None)
        ram = psutil.virtual_memory()
        if torch.cuda.is_available():
            gpu_mem = torch.cuda.memory_allocated() / 1024**2
        else:
            gpu_mem = 0.0
        print(f"[Batch {i+1}] Time: {time.time()-batch_start:.2f}s | CPU: {cpu_usage:.1f}% | RAM: {ram.used/1024**3:.2f}GB | GPU: {gpu_mem:.2f}MB")

end_time = time.time()

[Batch 1] Time: 0.04s | CPU: 15.7% | RAM: 45.28GB | GPU: 203.49MB
[Batch 2] Time: 0.01s | CPU: 98.7% | RAM: 45.31GB | GPU: 203.49MB
[Batch 3] Time: 0.01s | CPU: 92.3% | RAM: 45.32GB | GPU: 203.49MB
[Batch 4] Time: 0.01s | CPU: 98.1% | RAM: 45.32GB | GPU: 203.49MB
[Batch 5] Time: 0.01s | CPU: 87.6% | RAM: 45.31GB | GPU: 203.49MB
[Batch 6] Time: 0.01s | CPU: 62.8% | RAM: 45.36GB | GPU: 203.49MB
[Batch 7] Time: 0.01s | CPU: 58.9% | RAM: 45.30GB | GPU: 203.49MB
[Batch 8] Time: 0.01s | CPU: 51.3% | RAM: 45.30GB | GPU: 203.49MB
[Batch 9] Time: 0.01s | CPU: 49.8% | RAM: 45.34GB | GPU: 203.49MB
[Batch 10] Time: 0.01s | CPU: 59.8% | RAM: 45.34GB | GPU: 203.49MB
[Batch 11] Time: 0.01s | CPU: 60.6% | RAM: 45.35GB | GPU: 203.49MB
[Batch 12] Time: 0.01s | CPU: 59.7% | RAM: 45.35GB | GPU: 203.49MB
[Batch 13] Time: 0.01s | CPU: 94.6% | RAM: 45.36GB | GPU: 203.49MB
[Batch 14] Time: 0.01s | CPU: 98.2% | RAM: 45.35GB | GPU: 203.49MB
[Batch 15] Time: 0.01s | CPU: 98.8% | RAM: 45.36GB | GPU: 203.49MB
[Bat

In [36]:
total_time = end_time - start_time
print(f"\nTemps Test Total: {total_time:.2f} sec")
print(f"Throughput: {len(test_dataset) / total_time:.2f} images/sec")


Temps Test Total: 10.74 sec
Throughput: 572.79 images/sec


In [37]:
# Rapport complet
print("=== Test Set Evaluation (Student) ===")
print(classification_report(all_labels, all_preds, target_names=class_names))

=== Test Set Evaluation (Student) ===
                                               precision    recall  f1-score   support

                           Apple___Apple_scab       1.00      1.00      1.00       100
                            Apple___Black_rot       1.00      1.00      1.00       100
                     Apple___Cedar_apple_rust       1.00      1.00      1.00       100
                              Apple___healthy       1.00      0.99      1.00       164
                    Background_without_leaves       0.97      1.00      0.99       114
                          Blueberry___healthy       1.00      1.00      1.00       150
                      Cherry___Powdery_mildew       1.00      0.99      1.00       105
                             Cherry___healthy       1.00      0.99      0.99       100
   Corn___Cercospora_leaf_spot Gray_leaf_spot       0.98      0.99      0.99       100
                           Corn___Common_rust       0.99      1.00      1.00       119
    